# Basic EM workflow 3 (Restaurants data set)

# Introduction

This IPython notebook explains a basic workflow two tables using *py_entitymatching*. Our goal is to come up with a workflow to match restaurants from Fodors and Zagat sites. Specifically, we want to maximize F1. The datasets contain information about the restaurants.

First, we need to import *py_entitymatching* package and other libraries as follows:

In [1]:
import sys
#sys.path.append('/Users/pradap/Documents/Research/Python-Package/anhaid/py_entitymatching/')

import py_entitymatching as em
import pandas as pd
import os

In [2]:
# Display the versions
print('python version: ' + sys.version )
print('pandas version: ' + pd.__version__ )
print('magellan version: ' + em.__version__ )

python version: 3.6.5 (default, Apr  1 2018, 05:46:30) 
[GCC 7.3.0]
pandas version: 0.22.0
magellan version: 0.3.0


Matching two tables typically consists of the following three steps:

** 1. Reading the input tables **

** 2. Blocking the input tables to get a candidate set **

** 3. Matching the tuple pairs in the candidate set **

# Read input tables

We begin by loading the input tables. For the purpose of this guide, we use the datasets that are included with the package.

In [3]:
# Get the paths
path_A = '/home/liang/Workspace/cs839-project/stage-2/crawlers/data/amazon_products.csv'
path_B = '/home/liang/Workspace/cs839-project/stage-2/crawlers/data/walmart_products.csv'

In [4]:
# Load csv files as dataframes and set the key attribute in the dataframe
A = em.read_csv_metadata(path_A, key='id')
B = em.read_csv_metadata(path_B, key='id')

In [5]:
print('Number of tuples in A: ' + str(len(A)))
print('Number of tuples in B: ' + str(len(B)))
print('Number of tuples in A X B (i.e the cartesian product): ' + str(len(A)*len(B)))

Number of tuples in A: 2976
Number of tuples in B: 4847
Number of tuples in A X B (i.e the cartesian product): 14424672


In [6]:
A.head(2)

,id,product title,brand,model,operating system,combo
0,a0,acer predator helios 300 gaming laptop 15.6 full hd intel core i7-7700hq cpu 16gb ddr4 ram 256gb...,acer,nh.q28aa.001,windows 10,acer predator helios 300 gaming laptop 15.6 full hd intel core i7-7700hq cpu 16gb ddr4 ram 256gb...
1,a1,acer aspire e 15 15.6 full hd 8th gen intel core i5-8250u geforce mx150 8gb ram memory 256gb ssd...,acer,e5-576g-5762,windows 10,acer aspire e 15 15.6 full hd 8th gen intel core i5-8250u geforce mx150 8gb ram memory 256gb ssd...


In [7]:
B.head(2)

,id,product title,brand,model,operating system,combo
0,w0,iview i896qw 8.95 2-in-1 32gb tablet intel atom bay trail z3735f processor windows 10,iview,NaN,microsoft windows,iview i896qw 8.95 2-in-1 32gb tablet intel atom bay trail z3735f processor windows 10 iview micr...
1,w1,dell - inspiron 2-in-1 11.6 touch-screen laptop - intel pentium - 4gb memory - 500gb hd - red,dell,i3168-3270red,windows 10,dell - inspiron 2-in-1 11.6 touch-screen laptop - intel pentium - 4gb memory - 500gb hd - red de...


In [8]:
# Display the keys of the input tables
em.get_key(A), em.get_key(B)

('id', 'id')

# Block tables to get candidate set

Before we do the matching, we would like to remove the obviously non-matching tuple pairs from the input tables. This would reduce the number of tuple pairs considered for matching.
*py_entitymatching* provides four different blockers: (1) attribute equivalence, (2) overlap, (3) rule-based, and (4) black-box. The user can mix and match these blockers to form a blocking sequence applied to input tables.

For the matching problem at hand, we know that two restaurants with different city names will not match. So we decide the apply blocking over names:

In [9]:
# Blocking plan

# A, B -- attribute equiv. blocker [city] --------------------|---> candidate set

# block on: brand, ram, OS?, model? 

In [10]:
ab = em.AttrEquivalenceBlocker()
#For the given two tables, we will assume that two persons with different zipcode values do not refer to the same real world person. So, we apply attribute equivalence blocking on zipcode. That is, we block all the tuple pairs that have different zipcodes.

# Use block_tables to apply blocking over two input tables.
C1 = ab.block_tables(A, B, 
                    l_block_attr='brand', r_block_attr='brand', 
                    l_output_attrs=['id','brand', 'combo'],
                    r_output_attrs=['id','brand', 'combo'],
                    n_jobs=-1)
len(C1)

1636110

In [11]:
C1.head()

,_id,ltable_id,rtable_id,ltable_brand,ltable_combo,rtable_brand,rtable_combo
0,0,a0,w28,acer,acer predator helios 300 gaming laptop 15.6 full hd intel core i7-7700hq cpu 16gb ddr4 ram 256gb...,acer,manufacturer refurbished acer 11.6 intel celeron 1.1 ghz 4 gb ram 32 gb flash windows 10 home|sp...
1,1,a0,w30,acer,acer predator helios 300 gaming laptop 15.6 full hd intel core i7-7700hq cpu 16gb ddr4 ram 256gb...,acer,switch alpha 12 sa5-271-78m8 12in 8gb 256gb w10h 11ac i7-6500u acer nt.lcdaa.014 windows 10
2,2,a0,w48,acer,acer predator helios 300 gaming laptop 15.6 full hd intel core i7-7700hq cpu 16gb ddr4 ram 256gb...,acer,acer switch alpha 12 sa5-271p-5972 - 12 - core i5 6200u - 8 gb ram - 256 gb ssd - us internation...
3,3,a0,w49,acer,acer predator helios 300 gaming laptop 15.6 full hd intel core i7-7700hq cpu 16gb ddr4 ram 256gb...,acer,r5-471t-78vy i7-6500u 2.5g 8gb 256gb ssd 14in 11ac w10h acer nx.g7waa.012 windows 10
4,4,a0,w70,acer,acer predator helios 300 gaming laptop 15.6 full hd intel core i7-7700hq cpu 16gb ddr4 ram 256gb...,acer,acer switch alpha 12 sa5-271p-74e1 - 12 - core i7 6500u - 8 gb ram - 256 gb ssd - us internation...


## Debug blocker output

The number of tuple pairs considered for matching is reduced to 10165 (from 176423), but we would want to make sure that the blocker did not drop any potential matches. We could debug the blocker output in *py_entitymatching* as follows:

In [12]:
# # Debug blocker output
# dbg = em.debug_blocker(C1, A, B, output_size=10, 
#                        attr_corres=[
#                            ('product title','product title'),
#                            ('brand', 'brand'), 
#                            ('model','model'),
#                            ('combo','combo')],
#                       verbose=True)
# #### Display first few tuple pairs from the debug_blocker's output
# dbg.head()

From the debug blocker's output we observe that the current blocker drops quite a few potential matches. We would want to update the blocking sequence to avoid dropping these potential matches.

For the considered dataset, we know that for the restaurants to match the  names must overlap between them. We could use overlap blocker for this purpose. Finally, we would want to union the outputs from the attribute equivalence blocker and the overlap blocker to get a consolidated candidate set.

In [13]:
# Updated blocking sequence
# A, B ------ attribute equivalence [city] -----> C1--
#                                                     |----> C
# A, B ------ overlap blocker [name] --------> C2--

In [14]:
# Create overlap blocker
ob = em.OverlapBlocker()

# Block tables using 'name' attribute 
C2 = ob.block_tables(A,B, 'model', 'model', 
                    l_output_attrs=['id','model','combo'], 
                    r_output_attrs=['id','model','combo'],
                    overlap_size=8,
                    q_val=5,
                    word_level=False,
                    show_progress=True,
                    n_jobs=-1
                    )
len(C2)

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


2582

In [15]:
# Display first two rows from C2
C2.head()

,_id,ltable_id,rtable_id,ltable_model,ltable_combo,rtable_model,rtable_combo
0,0,a229,w1,i3168-3271blu,dell i3168-3271blu 11.6 hd 2-in-1 laptop (intel pentium n3710 1.6ghz processor 4 gb ddr3l sdram ...,i3168-3270red,dell - inspiron 2-in-1 11.6 touch-screen laptop - intel pentium - 4gb memory - 500gb hd - red de...
1,1,a1922,w7,i5578-5902gry,dell i5578-5902gry inspiron pro 15.6 fhd with touch laptop computer (core i5-7200u 8gb ddr4 256g...,i5578-3093gry,dell - inspiron 15 5000 2-in-1 15.6 touch-screen laptop - intel core i3 - 4gb memory - 500gb 540...
2,2,a229,w12,i3168-3271blu,dell i3168-3271blu 11.6 hd 2-in-1 laptop (intel pentium n3710 1.6ghz processor 4 gb ddr3l sdram ...,i3168-3271blu,dell - inspiron 11 3000 2-in-1 blue 11.6-inch hd intel pentium processor n3710 4gb 1600mhz ddr3l...
3,3,a1561,w15,i5579-7050gry,dell inspiron 13 i5379-7302gry-pus 2 in 1 pc i5579-7050gry windows 10,i5578-2550gry,dell - inspiron 15 5000 2-in-1 15.6-inch fhd intel core i7-7500u 8gb 2400mhz ddr4 1tb 5400 rpm h...
4,4,a1922,w15,i5578-5902gry,dell i5578-5902gry inspiron pro 15.6 fhd with touch laptop computer (core i5-7200u 8gb ddr4 256g...,i5578-2550gry,dell - inspiron 15 5000 2-in-1 15.6-inch fhd intel core i7-7500u 8gb 2400mhz ddr4 1tb 5400 rpm h...


In [16]:
# # Create overlap blocker
# ob = em.OverlapBlocker()

# # Block tables using 'name' attribute 
# C3 = ob.block_tables(A,B, 'ram size', 'ram size', 
#                     l_output_attrs=['id','ram size','combo'], 
#                     r_output_attrs=['id','ram size','combo'],
#                     overlap_size=5,
#                     q_val=5,
#                     word_level=False,
#                     show_progress=True,
#                     n_jobs=-1
#                     )
# len(C3)

In [17]:
# C3.head()

We add another blocker.

In [18]:
# Create overlap blocker
ob = em.OverlapBlocker()

# Block tables using 'name' attribute 
C4 = ob.block_tables(A, B, 'combo', 'combo', 
                    l_output_attrs=['id','combo'], 
                    r_output_attrs=['id','combo'],
                    overlap_size=80,
                    q_val=5,
                    word_level=False,
                    show_progress=True,
                    n_jobs=-1
                    )
len(C4)

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:03


5713

In [19]:
C4.head()

,_id,ltable_id,rtable_id,ltable_combo,rtable_combo
0,0,a322,w32,lenovo flex 5 2-in-1 laptop: core i5-8250u 8gb ram 128gb ssd 14-inch full hd touch display windo...,lenovo flex 5 2-in-1 laptop: core i5-8250u 8gb ram 128gb ssd 14-inch full hd touch display windo...
1,1,a255,w41,asus vivobook flip tp200sa-dh01t 11.6 inch display thin and lightweight 2-in-1 hd touchscreen la...,asus vivobook tp200sa-dh01t-bl 11.6 inch display thin and lightweight 2-in-1 hd touchscreen lapt...
2,2,a621,w41,asus vivobook flip 14 tp401ca-dhm6t 14” thin and lightweight 2-in-1 hd touchscreen laptop intel ...,asus vivobook tp200sa-dh01t-bl 11.6 inch display thin and lightweight 2-in-1 hd touchscreen lapt...
3,3,a648,w41,asus transformer book tp200sa-dh01t-bl 11.6 inch display thin and lightweight 2-in-1 full hd tou...,asus vivobook tp200sa-dh01t-bl 11.6 inch display thin and lightweight 2-in-1 hd touchscreen lapt...
4,4,a432,w72,dell inspiron 13 2-in-1 laptop: core i7-8550u 256gb ssd 8gb ram 13.3 full hd touch display windo...,dell inspiron 13 2-in-1 laptop: core i7-8550u 256gb ssd 8gb ram 13.3 full hd touch display dell ...


In [20]:
# s1 = pd.merge(C1, C3, how='inner', on=['ltable_id','rtable_id'], suffixes=('',''))
# len(s1)

In [21]:
# s1.head()

In [22]:
# Combine blocker outputs
C = em.combine_blocker_outputs_via_union([C2,C4])
len(C)

8158

In [23]:
C.head()

,_id,ltable_id,rtable_id,ltable_model,ltable_combo,rtable_model,rtable_combo
0,0,a0,w1389,nh.q28aa.001,acer predator helios 300 gaming laptop 15.6 full hd intel core i7-7700hq cpu 16gb ddr4 ram 256gb...,nh.q1taa.001,refurbished acer 17.3 intel core i7 2.8 ghz 16 gb ram 1 tb hdd + 256 gb ssd windows 10 home acer...
1,1,a0,w1396,nh.q28aa.001,acer predator helios 300 gaming laptop 15.6 full hd intel core i7-7700hq cpu 16gb ddr4 ram 256gb...,nh.q1aaa.001,manufacturer refurbished acer predator g9-793-79pe 17.3 intel core i7-6700hq 2.6 ghz 16gb ram 1t...
2,2,a0,w1504,nh.q28aa.001,acer predator helios 300 gaming laptop 15.6 full hd intel core i7-7700hq cpu 16gb ddr4 ram 256gb...,nh.q1faa.001,refurbished acer 17.3 intel core i7 2.9 ghz 32 gb ram 1 tb hdd + 512 gb ssd windows 10 home acer...
3,3,a0,w1516,nh.q28aa.001,acer predator helios 300 gaming laptop 15.6 full hd intel core i7-7700hq cpu 16gb ddr4 ram 256gb...,nh.q1eaa.001,refurbished acer 17.3 intel core i7 2.9 ghz 32 gb ram 1 tb hdd + 512 gb ssd windows 10 home acer...
4,4,a0,w1661,nh.q28aa.001,acer predator helios 300 gaming laptop 15.6 full hd intel core i7-7700hq cpu 16gb ddr4 ram 256gb...,nh.q13aa.001,refurbished acer 17.3 intel core i7 2.7 ghz 32 gb ram 1 tb hdd windows 10 home|gx-791-758v acer ...


We observe that the number of tuple pairs considered for matching is increased to 12530 (from 10165). Now let us debug the blocker output again to check if the current blocker sequence is dropping any potential matches.

In [24]:
# Debug again
#dbg = em.debug_blocker(C, A, B, output_size=10)

In [25]:
#dbg[['ltable_combo','rtable_combo']].to_csv('dbg.csv')


We observe that the current blocker sequence does not drop obvious potential matches, and we can proceed with the matching step now. A subtle point to note here is, debugging blocker output practically provides a stopping criteria for modifying the blocker sequence.


# Matching tuple pairs in the candidate set

In this step, we would want to match the tuple pairs in the candidate set. Specifically, we use learning-based method for matching purposes.
This typically involves the following five steps:
1. Sampling and labeling the candidate set
2. Splitting the labeled data into development and evaluation set
3. Selecting the best learning based matcher using the development set
4. Evaluating the selected matcher using the evaluation set

## Sampling and labeling the candidate set

First, we randomly sample 450 tuple pairs for labeling purposes.

In [26]:
# Sample  candidate set
# S = em.sample_table(C, 300)
# em.to_csv_metadata(S, '/home/liang/Workspace/cs839-project/stage-3/s.csv')
#S.head()

Next, we label the sampled candidate set. Specify we would enter 1 for a match and 0 for a non-match.

In [27]:
# Label S
# G = em.label_table(S, 'match')

For the purposes of this guide, we will load in a pre-labeled dataset (of 450 tuple pairs) included in this package.

In [28]:
# # Load the pre-labeled data
path_G = '/home/liang/Workspace/cs839-project/stage-3/s1.csv'
G = em.read_csv_metadata(path_G, 
                         key='_id',
                         ltable=A, rtable=B, 
                         fk_ltable='ltable_id', fk_rtable='rtable_id')
len(G)

300

## Splitting the labeled data into development and evaluation set

In this step, we split the labeled data into two sets: development (I) and evaluation (J). Specifically, the development set is used to come up with the best learning-based matcher and the evaluation set used to evaluate the selected matcher on unseen data.

In [29]:
# Split S into development set (I) and evaluation set (J)
IJ = em.split_train_test(G, train_proportion=0.7, random_state=0)
I = IJ['train']
J = IJ['test']

In [30]:
G.head(3)


,Unnamed: 0,_id,ltable_id,rtable_id,ltable_model,ltable_combo,rtable_model,rtable_combo,match
0,27,27,a1011,w685,t8tjg,2018 flagship lenovo thinkpad 11e (4h generation) 11.6 notebook intel i3-7100u 128gb m.2 ssd 4gb...,t8tjg,dell chromebook 11 3189 - 11.6 - celeron n3060 - 4 gb ram - 64 gb ssd dell t8tjg chrome os,0
1,63,63,a1044,w915,80xb000bus,lenovo flex 5 15.6 full hd touch multimode 2-in-1 notebook computer intel core i7-7500u 16gb ram...,80ys000bus,topseller n23 n3060 1.60g 4gb 32gb 11.6in hd display chrome lenovo 80ys000bus chrome os,0
2,99,99,a1047,w1550,n850hp6,prostar clevo gaming laptop n850hp6 15.6” fhd ips (1920x1080) led matte type display intel core ...,n850hk1,prostar clevo gaming laptop n850hk1 15.6” fhd ips (1920x1080) matte type display intel core i7-7...,1


## Selecting the best learning-based matcher 

Selecting the best learning-based matcher typically involves the following steps:

1. Creating a set of learning-based matchers
2. Creating features
3. Converting the development set into feature vectors
4. Selecting the best learning-based matcher using k-fold cross validation

### Creating a set of learning-based matchers

In [31]:
# Create a set of ML-matchers
dt = em.DTMatcher(name='DecisionTree', random_state=0)
svm = em.SVMMatcher(name='SVM', random_state=0)
rf = em.RFMatcher(name='RF', random_state=0)
lg = em.LogRegMatcher(name='LogReg', random_state=0)
ln = em.LinRegMatcher(name='LinReg')
nb = em.NBMatcher(name='NaiveBayes')

### Creating features

Next, we need to create a set of features for the development set. *py_entitymatching* provides a way to automatically generate features based on the attributes in the input tables. For the purposes of this guide, we use the automatically generated features.

In [32]:
match_t = em.get_tokenizers_for_matching()
match_s = em.get_sim_funs_for_matching()
atypes1 = em.get_attr_types(A) # don't need, if atypes1 exists from blocking step
atypes2 = em.get_attr_types(B) # don't need, if atypes2 exists from blocking step
match_c = em.get_attr_corres(A, B)
feature_table = em.get_features(A, B, atypes1, atypes2, match_c, match_t, match_s)

In [33]:
# Generate features
# feature_table = em.get_features_for_matching(A, B, validate_inferred_attr_types=True)
# feature_table = em.get_features(A, B, 
#                                 em.get_attr_types(A), em.get_attr_types(B), 
#                                 attr_corres, 
#                                 tok_funcs, 
#                                 sim_funcs)

In [34]:
# List the names of the features generated
feature_table['feature_name']

0                                            id_id_lev_dist
1                                             id_id_lev_sim
2                                                 id_id_jar
3                                                 id_id_jwn
4                                                 id_id_exm
5                                     id_id_jac_qgm_3_qgm_3
6               product_title_product_title_jac_qgm_3_qgm_3
7           product_title_product_title_cos_dlm_dc0_dlm_dc0
8                               brand_brand_jac_qgm_3_qgm_3
9                           brand_brand_cos_dlm_dc0_dlm_dc0
10                          brand_brand_jac_dlm_dc0_dlm_dc0
11                                          brand_brand_mel
12                                     brand_brand_lev_dist
13                                      brand_brand_lev_sim
14                                          brand_brand_nmw
15                                           brand_brand_sw
16                              model_mo

### Converting the development set to feature vectors

In [35]:
# Convert the I into a set of feature vectors using F
H = em.extract_feature_vecs(I, 
                            feature_table=feature_table, 
                            attrs_after='match',
                            show_progress=False)
# em.to_csv_metadata(H,'h.csv')

In [36]:
# Display first few rows
# H = em.read_csv_metadata('h.csv', 
#                          key='_id',
#                          ltable=A, rtable=B, 
#                          fk_ltable='ltable_id', fk_rtable='rtable_id')
H.fillna(0, inplace=True)
H.head(3)

,_id,ltable_id,rtable_id,id_id_lev_dist,id_id_lev_sim,id_id_jar,id_id_jwn,id_id_exm,id_id_jac_qgm_3_qgm_3,product_title_product_title_jac_qgm_3_qgm_3,...,operating_system_operating_system_cos_dlm_dc0_dlm_dc0,operating_system_operating_system_jac_dlm_dc0_dlm_dc0,operating_system_operating_system_mel,operating_system_operating_system_lev_dist,operating_system_operating_system_lev_sim,operating_system_operating_system_nmw,operating_system_operating_system_sw,combo_combo_jac_qgm_3_qgm_3,combo_combo_cos_dlm_dc0_dlm_dc0,match
282,7036,a888,w3248,4,0.2,0.483333,0.483333,0,0.083333,0.504237,...,0.408248,0.25,0.892063,6.0,0.571429,3.0,8.0,0.506224,0.678064,0
44,1158,a1347,w3193,5,0.0,0.433333,0.433333,0,0.000000,0.607656,...,0.408248,0.25,0.892063,6.0,0.571429,3.0,8.0,0.600000,0.714742,1
272,6861,a816,w3647,5,0.0,0.483333,0.483333,0,0.000000,0.266667,...,0.666667,0.50,0.648810,12.0,0.142857,-4.0,4.0,0.236842,0.426401,1


### Selecting the best matcher using cross-validation

Now, we select the best matcher using k-fold cross-validation. For the purposes of this guide, we use five fold cross validation and use 'precision' and 'recall' metric to select the best matcher.

In [37]:
# Select the best ML matcher using CV
result = em.select_matcher([dt, rf, svm, ln, lg, nb], table=H, 
        exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'match'],
        k=5,
        target_attr='match', metric_to_select_matcher='f1', random_state=0)
result['cv_stats']

,Matcher,Average precision,Average recall,Average f1
0,DecisionTree,0.913298,0.872038,0.890183
1,RF,0.912697,0.893897,0.902237
2,SVM,0.781989,0.941075,0.850426
3,LinReg,0.835312,0.860320,0.845554
4,LogReg,0.841853,0.932401,0.882235
5,NaiveBayes,0.812980,0.792736,0.799680


### Debugging matcher

We observe that the best matcher is not maximizing F1. We debug the matcher to see what might be wrong.
To do this, first we split the feature vectors into train and test.

In [38]:
#  Split feature vectors into train and test
UV = em.split_train_test(H, train_proportion=0.5)
U = UV['train']
V = UV['test']

Next, we debug the matcher using GUI. For the purposes of this guide, we use random forest matcher for debugging purposes.

In [39]:
# Debug rf using GUI
# em.vis_debug_rf(rf, U, V, 
#         exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'match'],
#         target_attr='match')

From the GUI, we observe that phone numbers seem to be an important attribute, but they are in different format. Current features does not capture and adding a feature incorporating this difference in format can potentially improve 
the F1 numbers.

In [40]:
# def phone_phone_feature(ltuple, rtuple):
#     p1 = ltuple.phone
#     p2 = rtuple.phone
#     p1 = p1.replace('/','-')
#     p1 = p1.replace(' ','')
#     p2 = p2.replace('/','-')
#     p2 = p2.replace(' ','')    
#     if p1 == p2:
#         return 1.0
#     else:
#         return 0.0

In [41]:
# feature_table = em.get_features_for_matching(A, B)
# em.add_blackbox_feature(feature_table, 'phone_phone_feature', phone_phone_feature)

Now, we repeat extracting feature vectors (this time with updated feature table), imputing table and selecting the best matcher again using cross-validation.

In [42]:
H = em.extract_feature_vecs(I, feature_table=feature_table, attrs_after='match', show_progress=False)
H.fillna(0, inplace=True)

In [43]:
# Select the best ML matcher using CV
result = em.select_matcher([dt, rf, svm, ln, lg, nb], table=H, 
        exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'match'],
        k=5,
        target_attr='match', metric_to_select_matcher='precision', random_state=0)
result['cv_stats']

,Matcher,Average precision,Average recall,Average f1
0,DecisionTree,0.913298,0.872038,0.890183
1,RF,0.912697,0.893897,0.902237
2,SVM,0.781989,0.941075,0.850426
3,LinReg,0.835312,0.860320,0.845554
4,LogReg,0.841853,0.932401,0.882235
5,NaiveBayes,0.812980,0.792736,0.799680


Now, observe the best matcher is achieving a better F1. Let us stop here and  proceed on to evaluating the best matcher on the unseen data (the evaluation set).

##  Evaluating the matching output

Evaluating the matching outputs for the evaluation set typically involves the following four steps:
1. Converting the evaluation set to feature vectors
2. Training matcher using the feature vectors extracted from the development set
3. Predicting the evaluation set using the trained matcher
4. Evaluating the predicted matches

### Converting the evaluation set to  feature vectors

As before, we convert to the feature vectors (using the feature table and the evaluation set)

In [ ]:
# Convert J into a set of feature vectors using feature table
L = em.extract_feature_vecs(J, feature_table=feature_table,
                            attrs_after='match', show_progress=False)

### Training the selected matcher

Now, we train the matcher using all of the feature vectors from the development set. For the purposes of this guide we use random forest as the selected matcher.

In [ ]:
# Train using feature vectors from I 
rf.fit(table=H, 
       exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'match'], 
       target_attr='match')

### Predicting the matches

Next, we predict the matches for the evaluation set (using the feature vectors extracted from it).

In [ ]:
L.head()
L.fillna(0, inplace=True)

In [ ]:
# Predict on L 
predictions = rf.predict(table=L, exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'match'], 
              append=True, target_attr='predicted', inplace=False)

### Evaluating the predictions

Finally, we evaluate the accuracy of predicted outputs

In [ ]:
# Evaluate the predictions
eval_result = em.eval_matches(predictions, 'match', 'predicted')
em.print_eval_summary(eval_result)